In [ ]:
#installing ekphrasis
!pip install ekphrasis
#installing tweet-preprocessor
!pip install tweet-preprocessor


In [2]:
import numpy as np
import pandas as pd
import sklearn
import preprocessor as p
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import set_printoptions
from pandas import DataFrame
from sklearn import metrics
from collections import Counter
from sklearn.datasets import make_classification


#Single Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

#Ensemble Models
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

#Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#Libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
import re
re.compile('<title>(.*)</title>')


In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
val = pd.read_csv('/content/drive/My Drive/fake_news/Val.csv',encoding = "ISO-8859-1")
train = pd.read_csv('/content/drive/My Drive/fake_news/Train.csv',encoding = "ISO-8859-1")
test = pd.read_csv('/content/drive/My Drive/fake_news/test_with_labels.csv',encoding = "ISO-8859-1")

frames = [val, train, test]

df = pd.concat(frames)
df = df.reset_index(drop=True)



Mounted at /content/drive


In [5]:
for i,v in enumerate(df['tweet']):
     df.loc[i,'tweet'] = p.clean(v)

def preprocess_data(data):
 data = data.astype(str).str.replace('\d+', '')
 lower_text = data.str.lower()
 lemmatizer = nltk.stem.WordNetLemmatizer()
 w_tokenizer =  TweetTokenizer()
 
 def lemmatize_text(text):
  return [(lemmatizer.lemmatize(w)) for w \
                       in w_tokenizer.tokenize((text))]
 def remove_punctuation(words):
   new_words = []
   for word in words:
      new_word = re.sub(r'[^\w\s]', '', (word))
      if new_word != '':
         new_words.append(new_word)
   return new_words
 words = lower_text.apply(lemmatize_text)
 words = words.apply(remove_punctuation)
 return pd.DataFrame(words)
pre_tweets = preprocess_data(df['tweet'])
df['tweet'] = pre_tweets
stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'] .apply(lambda x: [item for item in \
                                    x if item not in stop_words])



In [6]:
df['tweet'] = df['tweet'].astype(str).str.replace('[', '')
df['tweet'] = df['tweet'].astype(str).str.replace(']', '')
features = df['tweet']
target = df['label'].map({'real': 1, 'fake':0})

trainx, testx, trainy, testy = train_test_split(features, target, test_size = 0.20, random_state = 50)



In [7]:
def TFIDF_verctorizer(trainx,testx):
    vectorizer =  TfidfVectorizer(max_features=5)
    trainx_con = vectorizer.fit_transform(trainx)
    testx_con = vectorizer.fit_transform(testx)
    return trainx_con,testx_con
    
trainx_con_TF_IDF,testx_con_TF_IDF=TFIDF_verctorizer(trainx,testx)
trainx_con_TF_IDF = trainx_con_TF_IDF.toarray()
testx_con_TF_IDF = testx_con_TF_IDF.toarray()


In [ ]:
names = []
accuracy_final = []


dict_classifiers = {
    "Logreg": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, kernel='linear'),
    "DT": DecisionTreeClassifier(),
    "NB": MultinomialNB(),
}

for name, model in dict_classifiers.items():
    model = model.fit(trainx_con_TF_IDF, trainy)
    result = model.predict(testx_con_TF_IDF)
    accuracy = accuracy_score(testy,result)*100
    names.append(name)
    accuracy_final.append(accuracy)
     
      
result_sm = pd.DataFrame(names, accuracy_final)
result_sm



In [ ]:
DT = DecisionTreeClassifier()
NB = MultinomialNB()

names = []
accuracy_final = []


dict_classifiers = {
    "RD": RandomForestClassifier ( n_estimators=100,max_depth=10, min_samples_leaf=1, min_samples_split= 2, random_state=50),
    "ET": ExtraTreesClassifier( n_estimators=100,max_depth=10,min_samples_leaf=1, random_state=10),
    "AdaB": AdaBoostClassifier(base_estimator = DT,n_estimators= 100,learning_rate=1,  random_state=40),
    "GB": GradientBoostingClassifier(n_estimators= 200,max_depth=5,  random_state=80),
    "XGBC":XGBClassifier(base_estimator = DT,n_estimators= 200,max_depth=10,subsample=0.8,  random_state=90)
    }

for name, model in dict_classifiers.items():
     model = model.fit(trainx_con_TF_IDF, trainy)
     result = model.predict(testx_con_TF_IDF)
     accuracy = accuracy_score(testy,result)*100
     names.append(name)
     accuracy_final.append(accuracy)
     
      
result_em = pd.DataFrame(names, accuracy_final)
result_em

